In [1]:
import pandas as pd
import numpy as np

In [2]:
d_gcs=pd.read_csv("gcs_scores_updated.csv")
d_gcs=d_gcs.drop(columns=['nursingchartcelltypecat','nursingchartcelltypevallabel'])
d_gcs=d_gcs.drop_duplicates()
d_gcs=d_gcs.rename(columns={'nursingchartentryoffset':'offset'})

In [3]:
labs=pd.read_csv("labs_updated_with_creatinine.csv")
labs=labs.rename(columns={'labresultoffset':'offset'})

In [4]:
cardiovas=pd.read_csv("drugate_norm_updated.csv")
cardiovas=cardiovas.drop(columns=['nursingchartvalue','SOFA_cardio'])

In [10]:
#First merge
labs_cardio=pd.merge(labs,cardiovas,how="outer",on=['patientunitstayid','offset']).drop_duplicates()
labs_cardio.to_csv("labs_cardio_interim.csv",index=False)

In [11]:
del labs
del cardiovas

In [19]:
training_build=pd.merge(labs_cardio,d_gcs,how="outer",on=['patientunitstayid','offset']).drop_duplicates()

In [21]:
training_build.loc[training_build['patientunitstayid']==141168]

,patientunitstayid,offset,paO2_FiO2,platelets_x_1000,total_bilirubin,urinary_creatinine,creatinine,Norepinephrine,Epinephrine,Dopamine,Dobutamine,GCS_Score
0,141168,516,NaN,209.0,2.6,NaN,1.95,NaN,NaN,NaN,NaN,NaN
1,141168,1133,NaN,213.0,4.1,NaN,2.30,NaN,NaN,NaN,NaN,NaN
2,141168,1701,NaN,NaN,NaN,173.12,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,1805,82.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,2010,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,141168,2026,136.0,187.0,5.2,NaN,2.95,NaN,NaN,NaN,NaN,NaN
6,141168,2141,121.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2910679,141168,126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2910680,141168,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2910681,141168,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#Initial length due to replication
len(training_build)

12739964

In [23]:
#To correct the replication of offsets for same patients
training_build=training_build.groupby(['patientunitstayid','offset'],as_index=False).max().drop_duplicates()

In [24]:
len(training_build)

5870689

In [25]:
training_build=training_build.groupby(['patientunitstayid'],as_index=False).apply(pd.DataFrame.sort_values,'offset').reset_index()
training_build=training_build.drop(columns=['level_0','level_1'])

In [35]:
training_build.head()

,patientunitstayid,offset,paO2_FiO2,platelets_x_1000,total_bilirubin,urinary_creatinine,creatinine,Norepinephrine,Epinephrine,Dopamine,Dobutamine,GCS_Score
0,141168,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141168,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,141168,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,141168,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,141168,66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
training_build.to_csv("training_table.csv",index=False)

In [30]:
#dropping rows where all the parameters are null
'''training_build_filtered=training_build.loc[~((training_build['paO2_FiO2']==np.nan) &
                  (training_build['platelets_x_1000']==np.nan) &
                  (training_build['total_bilirubin']==np.nan) &
                  (training_build['urinary_creatinine']==np.nan) &
                  (training_build['creatinine']==np.nan) &
                  (training_build['Norepinephrine']==np.nan) &
                  (training_build['Epinephrine']==np.nan) &
                  (training_build['Dopamine']==np.nan) &
                  (training_build['Dobutamine']==np.nan) &
                  (training_build['GCS_Score']==np.nan))]'''
'''training_build_filtered=training_build.dropna(subset=['paO2_FiO2','platelets_x_1000','total_bilirubin','urinary_creatinine','creatinine','Norepinephrine','Epinephrine','Dopamine','Dobutamine','GCS_Score'])'''

In [32]:
'''training_build_filtered = training_build[np.isfinite(training_build['paO2_FiO2']) & 
                                        np.isfinite(training_build['platelets_x_1000']) &
                                        np.isfinite(training_build['total_bilirubin']) &
                                        np.isfinite(training_build['urinary_creatinine']) &
                                        np.isfinite(training_build['creatinine']) &
                                        np.isfinite(training_build['Norepinephrine']) &
                                        np.isfinite(training_build['Epinephrine']) &
                                        np.isfinite(training_build['Dopamine']) &
                                        np.isfinite(training_build['Dobutamine']) &
                                        np.isfinite(training_build['GCS_Score']) ]'''